# Thomas Fire Analysis 

### Author: Kimberlee Wong
### Link to Website: https://kimberleewong.github.io/

## About

### Purpose
The following analysis was done in two-fold. Its purpose is to get a better understanding of the 2017 Thomas Fire that affected Ventura and Santa Barbara Counties. The first part shows the process of producing a graph that shows how the average air quality was affected, and the second part visualizes the region of fire through a false imagery map. 

### Highlights

- Using rioxr to import the landsat data and access its attribute
- Visualizing both true and false color images
- Combining two different datasets to make one map

### About the Data

The landsat data comes from Microsoft Planetary Computer Data Catalogue, and it is a simplified collection of colored bands. It was processed to remove data outside land and coarsen the spatial resolution. 

The Thomas Fire boundary was created in the previous notebook by filtering a California Fire perimeter file available from the US Government Data Catalogue.

### References

Microsoft Planetary Computer Data Catalogue, Landsat collection 2 Level-2 [Data file] Available from: https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2. Access date: November 23, 2024.


Data.gov Data Catalogue, California Fire Perimeters (all) [Data file] Available from: https://catalog.data.gov/dataset/california-fire-perimeters-all-b3436. Access date: November 23, 2024.

## Import Libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import rioxarray as rioxr
from shapely.geometry import box  # To create polygon bounding box

pd.set_option("display.max.columns", None) # To see all columns

## Load Data

In [ ]:
# 2017 and 2018 AQI Data
aqi_17 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip', compression = 'zip')
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip', compression = 'zip')

In [ ]:


# Read in data
aqi_17 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip', compression = 'zip')
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip', compression = 'zip')

# Combine data into one dataframe
aqi = pd.concat([aqi_17, aqi_18])

# Wrangle data 
aqi.columns = (aqi.columns
                  .str.lower()
                  .str.replace(' ','_')
                )

aqi_sb = aqi[aqi['county_name'] == 'Santa Barbara'].drop(columns = ['state_name', 'county_name', 'state_code', 'county_code'])


aqi_sb.date = pd.to_datetime(aqi_sb.date)

aqi_sb = aqi_sb.set_index('date')

aqi_sb['five_day_average'] = aqi_sb.rolling('5D').aqi.mean()   

# Visualize it
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(aqi_sb.index, aqi_sb.aqi, label='Daily AQI')
ax.plot(aqi_sb.index, aqi_sb.five_day_average, label='Five Day Average AQI')
ax.set_xlabel('Date')
ax.set_ylabel('AQI')
ax.set_title('Daily and Five Day Average AQI Readings of Santa Barbara County from 2017 to 2018')
ax.legend()